### Proceso Vertex Pipeline.

El proceso de construcción del pipeline en el script vertex_pipeline.py falla por permisos.

### Proceso de Cloud Function de Trigger.

In [1]:
from migracion_gcp.lib.clients.factory.abstract_factory import CloudFactory


functions_service = CloudFactory.create_function_service(cloud_provider = "gcp",
                                                         project_id = "mlops-credits-vertex-poc")

functions_service.create_function(fn_name = "trigger_reentrenamiento",
                                  trigger_type = "pubsub",
                                  entry_point = "main",
                                  trigger_topic_name = "consumers-iris-trigger-entrenamiento",
                                  source_archive_url = "gs://migracion-gcp-bucket/consumers/consumers_iris/MLB/1/trigger_reentrenamiento.zip")


## Faltan permisos para crear Cloud Functions.

2025-11-10 13:37:12,628 - INFO - Instanciando la primera instancia de la clase...
/Users/jusaldarriag/Library/Caches/pypoetry/virtualenvs/migracion-gcp-c2EB0SaH-py3.11/lib/python3.11/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
2025-11-10 13:37:13,240 - INFO - Esta clase implementa el patrón Singleton, por ende, se retorna la instancia existente.


InternalServerError: 500 Validation failed for trigger projects/mlops-credits-vertex-poc/locations/us-east1/triggers/trigger-reentrenamiento-386005: An internal error has occurred (f503c3a1-e0e4-426a-914e-d2ce2addfe8e)

In [3]:
from google.cloud import aiplatform


aiplatform.init(project="mlops-credits-vertex-poc", location="us-east1", experiment="mi-experimento")


# Crear un experimento forzará la creación del metadata store 'default'
with aiplatform.start_run("dummy-experiment"):
    aiplatform.log_params({"learning_rate": 0.001, "epochs": 10})
    aiplatform.log_metrics({"accuracy": 0.87})

Creating Tensorboard
Create Tensorboard backing LRO: projects/310075297167/locations/us-east1/tensorboards/1616545975621386240/operations/5278017621369815040
Tensorboard created. Resource name: projects/310075297167/locations/us-east1/tensorboards/1616545975621386240
To use this Tensorboard in another session:
tb = aiplatform.Tensorboard('projects/310075297167/locations/us-east1/tensorboards/1616545975621386240')


Associating projects/310075297167/locations/us-east1/metadataStores/default/contexts/mi-experimento-dummy-experiment to Experiment: mi-experimento


### Proceso Pub/Sub

In [1]:
from migracion_gcp.lib.clients.factory.abstract_factory import CloudFactory


pub_sub_service = CloudFactory.create_messaging_service(cloud_provider = "gcp",
                                                        project_id = "mlops-credits-vertex-poc")


pub_sub_service.create_topic(topic_name = "consumers-iris-trigger-entrenamiento")

2025-11-09 20:47:47,522 - INFO - Instanciando la primera instancia de la clase...
/Users/jusaldarriag/Library/Caches/pypoetry/virtualenvs/migracion-gcp-c2EB0SaH-py3.11/lib/python3.11/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/jusaldarriag/Library/Caches/pypoetry/virtualenvs/migracion-gcp-c2EB0SaH-py3.11/lib/python3.11/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authenticati

##### Batch Process.

In [5]:
pub_sub_service.batch_pull(sub_name = "streaming_sub", max_messages = 10)

2025-11-09 20:34:59,793 - INFO - Obteniendo mensajes de la suscripción: streaming_sub.


[]

##### Stream Process.

In [2]:
import json
import logging


logging.basicConfig(level = logging.INFO,
                    format = "%(asctime)s - %(levelname)s - %(message)s")


def callback(message):
    message_data = message.data.decode("utf-8")
    payload = json.loads(message_data)
    logging.info(f"Mensaje: {payload}, Tipo: {type(payload)}")

    message.ack()


streaming_pull_future = pub_sub_service.stream_pull(sub_name = "streaming_sub",
                                                    callback = callback)
streaming_pull_future.result()

2025-11-09 20:47:54,919 - INFO - Mensaje: {'message': 'Eso hpta!', 'escrito_por': '123'}, Tipo: <class 'dict'>
2025-11-09 20:48:00,520 - INFO - Mensaje: {'message': 'Eso hpta!', 'escrito_por': '123'}, Tipo: <class 'dict'>


KeyboardInterrupt: 